In [2]:
import uuid
import json
from os import path
from datetime import datetime, timedelta

In [3]:
def clean_gpt_geojson(gpt_output):
  def safe_int(val):
    try:
        return int(str(val).replace(",", "").strip())
    except:
        return None

  # Get today's date and 1 year prior
  end_date = datetime.now()
  begin_date = end_date - timedelta(days=365)

  # Format in ISO8601 with Zulu time (UTC)
  begin_date_str = begin_date.strftime('%Y-%m-%dT00:00:00Z')
  end_date_str = end_date.strftime('%Y-%m-%dT00:00:00Z')

  # Create final geojson object
  full_geojson = {
      "type": "FeatureCollection",
      "mappers": [],
      "project": {
          "id": str(uuid.uuid4()),
          "name": "Generated Project",
          "begin_date": begin_date_str,
          "end_date": end_date_str,
          "cec_climate_zone": None,
          "climate_zone": "4A",
          "default_template": "90.1-2013",
          "import_surrounding_buildings_as_shading": None,
          "surface_elevation": None,
          "tariff_filename": None,
          "timesteps_per_hour": 1,
          "weather_filename": "weather.epw"
      },
      "scenarios": [
          {
              "feature_mappings": [],
              "id": str(uuid.uuid4()),
              "name": "Base Scenario"
          }
      ],
      "features": [
          {
              "type": "Feature",
              "properties": {
                  "id": str(uuid.uuid4()),
                  "name": "Generated Home",
                  "type": "Building",
                  "building_type": "Single family",
                  "floor_area": safe_int(gpt_output["geojson"]["properties"].get("Total Square Feet Living Area")),
                  "number_of_stories": safe_int(gpt_output["geojson"]["properties"].get("Number of Stories")),
                  "inspection_note": gpt_output["inspection_note"],
                  **gpt_output["geojson"]["properties"]
              },
              "geometry": gpt_output["geojson"]["geometry"]
          }
      ]
  }

  return full_geojson

In [5]:
home = 'dataset/RAMBEAU_RD_15'

with open(path.join(home, 'preprocessed.json'), 'r') as f:
  gpt_output = json.load(f)
  cleaned = clean_gpt_geojson(gpt_output)

  with open(path.join(home, 'cleaned.geojson'), 'w') as out:
    json.dump(cleaned, out)

print("Successfully cleaned GPT GeoJSON output.")


Successfully cleaned GPT GeoJSON output.


In [ ]:
# TODO: Loop Through Dataset